In [ ]:
#抓取数据 完整版3
import requests
import pandas as pd
from datetime import datetime

# 设置请求头
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "Client-Version": "v2.47.69",
    "Cookie": "_s_tentry=-; Apache=7189147422434.545.1748520876739; SINAGLOBAL=7189147422434.545.1748520876739; ULV=1748520876740:1:1:1:7189147422434.545.1748520876739:; SCF=Am3ezcllLXGaIlKWM5BixhZayhZGksyNNA1ToG1Mx9HKcrYgUod8CWLMHJyAA2Hs05a7CGvWOtAwL6JV_YCKSsY.; SUB=_2A25FPD-4DeRhGeNJ6FQY9S7LyjqIHXVmMD1wrDV8PUNbmtANLRHjkW9NS8FwfEzCt0yRPFIOmo2nLSgcOpIqPE-e; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFYEqzzDD6OgY1QUPwxssGV5NHD95QfS0ec1K-7S02cWs4DqcjMi--NiK.Xi-2Ri--ciKnRi-zNSKM0So.fehMpSntt; ALF=02_1751112936; XSRF-TOKEN=z9v0J58LTqwSJHuxbhmUrvgs; WBPSESS=ihNrCUzIK5bMw3CG01-xjRiYg_zMKozVwaIClGCK1pOxv0g-W69ZWyZCmNHC7zQsq9refFNnw_JQBrBb1tcQKtbMz0-bzLG9SCp6OG5pEaHIaifO30vjINQxnEcJ0jZl6F7jKISpNt4LzsZijE6Cfw==",
    "Priority": "u=1, i",
    "Referer": "https://weibo.com/2656274875/PtZb7AkOV",
    "Sec-CH-UA": "\"Chromium\";v=\"136\", \"Microsoft Edge\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "Sec-CH-UA-Mobile": "?0",
    "Sec-CH-UA-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Server-Version": "v2025.05.29.1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0",
    "X-Requested-With": "XMLHttpRequest",
    "X-XSRF-TOKEN": "z9v0J58LTqwSJHuxbhmUrvgs"
}

# 定义目标 URL，替换为实际的微博评论 API 链接
url = "https://weibo.com/ajax/statuses/buildComments?is_reload=1&id=5171466858659845&is_show_bulletin=2&is_mix=0&count=10&uid=2656274875&fetch_level=0&locale=zh-CN"

# 设置 max_id 为 None（第一页）
max_id = None
all_comments = []

# 获取并分页获取评论
while True:
    # 在 URL 中添加 max_id 参数
    if max_id:
        page_url = f"{url}&max_id={max_id}"
    else:
        page_url = url
    
    # 发送 GET 请求
    response = requests.get(page_url, headers=headers)

    # 检查响应状态
    if response.status_code == 200:
        comments_data = response.json()

        # 获取当前页面的评论
        comments = comments_data.get("data", [])
        all_comments.extend(comments)

        # 获取下一页的 max_id（如果有的话）
        max_id = comments_data.get("max_id", None)

        # 如果 max_id 为空，表示没有更多评论，退出循环
        if not max_id:
            break
    else:
        print(f"请求失败，状态码: {response.status_code}")
        break

# 格式化评论时间并保存数据
comments_list = []

# 将评论数据整理为表格格式
for comment in all_comments:
    # 获取评论的时间并转换为更易读的格式
    created_at = comment.get("created_at", "未知时间")
    #try:
    #   # 将时间字符串转换为标准时间格式
    #    created_at = datetime.strptime(created_at, "%a %b %d %H:%M:%S +0800 %Y")
    #    created_at = created_at.strftime("%Y-%m-%d %H:%M:%S")
    #except ValueError:
    #    created_at = "时间格式错误"

    text = comment.get("text", "没有评论内容")
    user_name = comment['user'].get('screen_name', '未知用户')
    like_counts = comment.get("like_counts", 0)

    comments_list.append([created_at, user_name, text, like_counts])

# 设置保存路径（例如 D:/weibo_comments.csv）
save_path = "D:/weibo_comments.csv"  # 修改为你希望保存的路径

# 使用 pandas 将评论数据保存为 CSV 文件
df = pd.DataFrame(comments_list, columns=["评论时间", "评论者", "评论内容", "点赞数"])

# 保存为 CSV 文件
df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"评论数据已保存到 {save_path}")


In [ ]:
#数据预处理（评论清理、时间转换、表情填充文字）【新版】
import pandas as pd
import re
from datetime import datetime

# 表情符号与文字的映射关系
emoji_dict = {
    "[威武]": "威武",
    "[666]": "666",
    "[赞]": "赞",
    "[打call]": "打call",
    "[送花花]": "送花花",
    "[good]": "good",
    "[素龙比心]": "素龙比心",
    "[相爱]": "相爱",
    "[努力]": "努力",
    "[awsl]": "awsl",
    "[鼓掌]": "鼓掌",
    "[酷]": "酷",
}

# 表情替换：提取<img>标签中的alt并替换
def replace_emojis_with_alt(text):
    # 替换<img ... alt="[表情]"...>为[表情]
    return re.sub(r'<img[^>]*alt="(\[.*?\])"[^>]*>', r'\1', text)

# 清理评论内容
def clean_text(text):
    if not isinstance(text, str):
        return "无效内容"
    
    # 保留超链接中文文本（如微博话题、用户）
    text = re.sub(r'<a[^>]*>(.*?)</a>', r'\1', text)

    # 提取alt里的表情标记
    text = replace_emojis_with_alt(text)

    # 替换表情标记为中文
    for emoji, word in emoji_dict.items():
        text = text.replace(emoji, word)

    # 删除剩余HTML标签
    text = re.sub(r'<.*?>', '', text)

    return text.strip()

# 评论时间格式转换
def convert_time(time_str):
    try:
        time_obj = datetime.strptime(time_str, "%a %b %d %H:%M:%S +0800 %Y")
        return time_obj.strftime("%Y-%m-%d %H:%M:%S")
    except Exception:
        return "时间格式错误"

# 读取原始CSV
df = pd.read_csv("D:/weibo_comments.csv")

# 清洗内容列
df['评论内容'] = df['评论内容'].apply(clean_text)

# 清洗时间列
df['评论时间'] = df['评论时间'].apply(convert_time)

# 处理空内容
df['评论内容'] = df['评论内容'].replace("", "没有文字内容")

# 查看前几行结果
print(df.head())

# 保存处理后的结果
df.to_csv("D:/weibo_comments_cleaned.csv", index=False, encoding="utf-8-sig")
print("✅ 数据预处理完成，已保存到 D:/weibo_comments_cleaned.csv")